<a href="https://colab.research.google.com/github/Samar-mami18/SimpleAgent/blob/main/FirstAgentThabet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
transformers_version = "v4.29.0"

print(f"Setting up everything with transformers version {transformers_version}")

!pip install huggingface_hub>=0.14.1 git+https://github.com/huggingface/transformers@$transformers_version -q diffusers accelerate datasets torch soundfile sentencepiece opencv-python openai
!pip install beautifulsoup4
!pip install requests

Setting up everything with transformers version v4.29.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [29]:
from transformers import Tool
import requests
from bs4 import BeautifulSoup

class TechTracker(Tool):
  name = "Hacker_Scraper"
  description = ("This is a tool that scraps the Hacker News website. It takes ino input and returns a list of tuples named `titles_links` containing the top scored websites titles and links.")
  inputs=[]
  outputs=["text"]

  def __call__(self):
    res = requests.get("https://news.ycombinator.com/news")
    soup = BeautifulSoup(res.text, "html.parser")

    titlelines = soup.select(".titleline")
    scores = soup.select(".score")

    scores = [int(score.contents[0].split(' ')[0]) for score in scores]
    titles_links = [(titlelines[i].contents[0].contents[0], titlelines[i].contents[0].get('href')) for i,score in enumerate(scores) if score>100]
    return titles_links

In [30]:
hacker_scraper_tool = TechTracker()

In [31]:
from transformers.tools import HfAgent

agent = HfAgent("https://api-inference.huggingface.co/models/bigcode/starcoder", additional_tools=[hacker_scraper_tool])

In [32]:
agent.toolbox

{'document_qa': PreTool(task='document-question-answering', description='This is a tool that answers a question about an document (pdf). It takes an input named `document` which should be the document containing the information, as well as a `question` that is the question about the document. It returns a text that contains the answer to the question.', repo_id=None),
 'image_captioner': PreTool(task='image-captioning', description='This is a tool that generates a description of an image. It takes an input named `image` which should be the image to caption, and returns a text that contains the description in English.', repo_id=None),
 'image_qa': PreTool(task='image-question-answering', description='This is a tool that answers a question about an image. It takes an input named `image` which should be the image containing the information, as well as a `question` which should be the question in English. It returns a text that is the answer to the question.', repo_id=None),
 'image_segmen

In [33]:
agent.run("scrap the Hacker News website")

==Explanation from the agent==
I will use the following  tool: `Hacker_Scraper` to scrap the website.


==Code generated by the agent==
titles_links = Hacker_Scraper()


==Result==


[('What Wikipedia Did',
  'https://twitter.com/jimmy_wales/status/1657494022741426180'),
 ("Instead of your Life's Purpose (2021)",
  'https://philosophyinhell.substack.com/p/instead-of-your-lifes-purpose'),
 ('Three Companies Impersonated Millions to Influence Internet Policy',
  'https://ag.ny.gov/press-release/2023/attorney-general-james-secures-615000-companies-supplied-fake-comments-influence'),
 ('Hundreds of millions of stars turned into a map of GitHub projects',
  'https://anvaka.github.io/map-of-github/'),
 ('macOS Apps in Rust', 'https://github.com/ryanmcgrath/cacao'),
 ('Idle DR-DOS', 'http://www.os2museum.com/wp/idle-dr-dos/'),
 ('Gamedev from Scratch 1: Scaffolding',
  'https://eev.ee/blog/2021/01/26/gamedev-from-scratch-1-scaffolding/'),
 ('Large language models generate functional protein sequences across families',
  'https://www.nature.com/articles/s41587-022-01618-2'),
 ('DIY Git in Python', 'https://www.leshenko.net/p/ugit/'),
 ('Vimcasts – Free screencasts about th

In [ ]:
texts = [BeautifulSoup(requests.get(title_link[1]).text, "html.parser").get_text() for title_link in titles_links if BeautifulSoup(requests.get(title_link[1]).text, "html.parser").get_text().find("JavaScript is not available")==-1]
